In [1]:
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

from unet.unet_model import *
from models import *
from datasets import *
from utils import *

import torch.nn as nn
import torch.nn.functional as F
import torch

from dataset.time_dataset import NewTsDataset
from pathlib import Path

import matplotlib.pyplot as plt

from dtw import dtw

In [2]:
# Dataset loader
datapath = Path('data')
dataset = NewTsDataset(datapath/'CaseI-Attacks without any change.csv')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                        shuffle=True, num_workers=4)
print("The size of the dataset is: ", dataset.data_normal.size(), dataset.data_abnormal.size())

The size of the dataset is:  torch.Size([33, 1, 576]) torch.Size([33, 1, 576])


In [4]:
normal_feature_len = dataset.data_normal.size(2)
abnormal_feature_len = dataset.data_abnormal.size(2)
normal_seq_len = dataset.data_normal.size(1)
abnormal_seq_len = dataset.data_abnormal.size(1)
normal_feature_len, abnormal_feature_len, normal_seq_len, abnormal_seq_len

(576, 576, 1, 1)

In [5]:
G_AB = LSTMUnetGenerator(33, normal_seq_len, normal_feature_len)
G_BA = LSTMUnetGenerator(33, abnormal_seq_len, abnormal_feature_len)
D_A = LSTMDiscriminator(normal_feature_len)
D_B = LSTMDiscriminator(abnormal_feature_len)

In [6]:
G_AB.load_state_dict(torch.load("saved_models/cyber_new/G_AB_190.pth"))
G_BA.load_state_dict(torch.load("saved_models/cyber_new/G_BA_190.pth"))
D_A.load_state_dict(torch.load("saved_models/cyber_new/D_A_190.pth"))
D_B.load_state_dict(torch.load("saved_models/cyber_new/D_B_190.pth"))

<All keys matched successfully>

In [7]:
G_AB.eval()
G_BA.eval()
D_A.eval()
D_B.eval()

LSTMDiscriminator(
  (lstm): LSTM(576, 256, batch_first=True)
  (linear): Sequential(
    (0): Linear(in_features=256, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [8]:
dataset.data_normal.size()

torch.Size([33, 1, 576])

In [9]:
threshold = 6.0

In [10]:
dataset_test = NewTsDataset(datapath/'CaseII.csv')
dataset_test.data_normal.size()

torch.Size([33, 1, 144])

In [11]:
dataset_test.data_normal.repeat(1, 1, 4).shape

torch.Size([33, 1, 576])

In [14]:
# Assuming dataset_test.data_normal.repeat(1, 1, 4) is the tensor you want to add noise to
noisy_tensor = dataset_test.data_normal.repeat(1, 1, 4) + torch.randn(dataset_test.data_normal.repeat(1, 1, 4).shape)

# Print the shape of the noisy tensor
print(noisy_tensor.shape)

torch.Size([33, 1, 576])


In [15]:
output = G_AB(noisy_tensor)
output_back = G_BA(output)
output.shape, output_back.shape

(torch.Size([33, 1, 576]), torch.Size([33, 1, 576]))

In [16]:
with torch.no_grad():
    output = G_AB(dataset_test.data_normal.repeat(1, 1, 4))
    output_back = G_BA(output)
print(f"Output size: {output.size()} \t Output Back size: {output_back.size()}")


for i in range(output.size(0)):
    plt.plot(output[i, 0, :].numpy())
    plt.plot(output_back[i, 0, :].numpy())
    plt.plot(dataset_test.data_normal[i, 0, :].numpy())
    plt.legend(['Generated', 'Generated Back', 'Original'])
    plt.savefig(f'cyber_result/new/forward/output_{i}.png')
    plt.close()

    dist1, cost, acc, path = dtw(output[i,0,:].reshape(-1, 1), dataset_test.data_normal[i,0,:].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    dist2, cost, acc, path = dtw(output_back[i,0,:].reshape(-1, 1), output[i,0,:].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    dist3, cost, acc, path = dtw(output_back[i,0,:].reshape(-1, 1), dataset_test.data_normal[i,0,:].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    res = (dist2+dist3)/dist1
    print("+" if res < threshold else "-", res)

Output size: torch.Size([33, 1, 576]) 	 Output Back size: torch.Size([33, 1, 576])
- 8.77335820515503
- 8.248511232425786
- 8.649096214490402
- 9.240148102431487


KeyboardInterrupt: 

---

In [ ]:
dataset_test.data_abnormal.size()

torch.Size([33, 1, 144])

In [ ]:
data_abnormal_test = dataset.data_abnormal[:,:,:]
data_abnormal_test.size()

torch.Size([33, 1, 576])

In [ ]:
with torch.no_grad():
    output = G_AB(data_abnormal_test)
    output_back = G_BA(output)

In [ ]:
for i in range(output.size(0)):
    plt.plot(output[i, 0, :].numpy())
    plt.plot(output_back[i, 0, :].numpy())
    plt.plot(data_abnormal_test[i, 0, :].numpy())
    plt.legend(['Generated', 'Generated Back', 'Original'])
    plt.savefig(f'cyber_result/detection/output_{i}.png')
    plt.close()

    dist1, cost, acc, path = dtw(output[i,0,:].reshape(-1, 1), data_abnormal_test[i,0,:].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    dist2, cost, acc, path = dtw(output_back[i,0,:].reshape(-1, 1), output[i,0,:].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    dist3, cost, acc, path = dtw(output_back[i,0,:].reshape(-1, 1), data_abnormal_test[i,0,:].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1)) 
    res = (dist2+dist3)/dist1   
    print("+" if res < threshold else "-", res)

- 7.783490175983338
+ 5.22026952259301
+ 5.554223316149699
- 7.704351576312942
- 11.423459216250198
- 6.643061377670183
- 6.643061377670183
- 17.984224235383753
- 18.353292812651002
- 17.89049843925211
- 18.05285578440196
- 17.480118543201513
- 12.972129353480696
+ 1.0232251716045897
+ 2.069967104175299
+ 1.7359055197613948
+ 1.2400592506194859
+ 2.161298658520614
- 21.98135402163749
+ 1.3208503536944418
+ 4.025003158882967
+ 1.200075569396496
+ 1.2931424043742938
+ 2.9265855922844684
+ 1.6697872308373367
- 6.447818380783491
- 9.704022919896826
+ 4.025003158882967
+ 1.9377114825577808
+ 0.7512348216120813
+ 4.871529006170861
- 6.890443803395747
+ 4.2408844451976275
